In [2]:
!nvidia-smi

Sun Oct 15 09:22:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import numpy as np
import cv2
import glob
import os

from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [30]:
DATA_DIR = "/kaggle/working/data/train/"
VAL_DATA_DIR = "/kaggle/working/data/validation/"

# DATA_DIR = "/kaggle/input/face-expression-recognition-dataset/images/train"
# VAL_DATA_DIR = "/kaggle/input/face-expression-recognition-dataset/images/validation"

BATCH_SIZE = 16
EPOCHS = 8
# WORKERS = 0 if os.name == 'nt' else 8
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(DEVICE))

Running on device: cuda


In [31]:
!mkdir data
!mkdir data/train
!mkdir data/validation

labels = ["happy", "neutral", "sad"]
split = ["train", "validation"]
for s in split:
    for l in labels:
        os.mkdir(f"/kaggle/working/data/{s}/{l}")

In [33]:
labels = ["happy", "neutral", "sad"]
split = ["train", "validation"]

for s in split:
    for l in labels:
        imgs = glob.glob(f'/kaggle/input/face-expression-recognition-dataset/images/{s}/{l}/*')
        for img in imgs:
            img_name = img.split("/")[-1]
            path = f"/kaggle/working/data/{s}/{l}/{img_name}"
            img = Image.open(img)
#             img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
#             img = Image.fromarray(img)
            img.save(path)

In [39]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=3
)

for p in resnet.parameters():
    p.requires_grad = False

for p in resnet.last_linear.parameters():
    p.requires_grad = True

for p in resnet.last_bn.parameters():
    p.requires_grad = True

resnet.logits = torch.nn.Linear(512, 3)
resnet = resnet.to(DEVICE)

optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
#     np.float32,
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    fixed_image_standardization
])

dataset = datasets.ImageFolder(DATA_DIR, transform=trans)
val_dataset = datasets.ImageFolder(VAL_DATA_DIR, transform=trans)

train_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
)

loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [40]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=DEVICE,
)

for epoch in range(EPOCHS):
    print('\nEpoch {}/{}'.format(epoch + 1, EPOCHS))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=DEVICE,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=DEVICE,
        writer=writer
    )




Initial
----------
Valid |   262/262  | loss:    1.0820 | fps:   65.9298 | acc:    0.4354   

Epoch 1/8
----------
Train |  1068/1068 | loss:    0.2423 | fps:   64.5225 | acc:    0.9366   
Valid |   262/262  | loss:    5.6598 | fps:   66.2953 | acc:    0.2746   

Epoch 2/8
----------
Train |  1068/1068 | loss:    0.2850 | fps:   64.5473 | acc:    0.9340   
Valid |   262/262  | loss:    4.9646 | fps:   66.1259 | acc:    0.2746   

Epoch 3/8
----------
Train |  1068/1068 | loss:    0.3003 | fps:   64.4826 | acc:    0.9253   
Valid |   262/262  | loss:    5.1180 | fps:   66.0975 | acc:    0.2746   

Epoch 4/8
----------
Train |  1068/1068 | loss:    0.3033 | fps:   64.5854 | acc:    0.9252   
Valid |   262/262  | loss:    4.7563 | fps:   66.0865 | acc:    0.2746   

Epoch 5/8
----------
Train |  1068/1068 | loss:    0.2903 | fps:   64.5574 | acc:    0.9281   
Valid |   262/262  | loss:    4.7093 | fps:   66.1269 | acc:    0.2746   

Epoch 6/8
----------
Train |  1068/1068 | loss:    1.5

In [41]:
!nvidia-smi

Sun Oct 15 13:59:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    32W / 250W |  12809MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [42]:
# save model
torch.save(resnet.state_dict(), 'finetuned_fer.pt')

hasilnya terlalu jelek